In [ ]:
# default_exp tests

# Test utils

This notebook contains common code used to test different things. 

In [ ]:
#export
import torch

In [ ]:
#hide


In [ ]:
#export 
def check_no_nans_or_infs(x):
    assert torch.all(~torch.isnan(x))
    assert torch.all(~torch.isneginf(x))
    assert torch.all(~torch.isposinf(x))

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 30_logging.ipynb.
Converted 35_charts.ipynb.
Converted index.ipynb.
Converted run.ipynb.
